# Universidad Autónoma de Nuevo León
# Facultad de Ciencias Físico Matemáticas
# Maestría en Ciencia de Datos

## Procesamiento y clasificación de datos

### Tarea 2

###### Jorge Loredo Gonzalez
###### Matricula: 1850739



#### Codigo

In [1]:
# https://github.com/raduangelescu/gutenbergpy
!pip install gutenbergpy
!pip install praw -q
!pip install wordcloud -q

In [2]:
import gutenbergpy.textget
import pandas as pd
import praw
import matplotlib.pyplot as plt
from wordcloud import WordCloud

import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.util import ngrams
from collections import Counter
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')

import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# Configuración de Reddit API
reddit = praw.Reddit(
    client_id="EEPZITq9hYh9vlgws1y5XQ",
    client_secret="gtMAaA2jExW07AnG3o93qZy21hdvFw",
    user_agent="explorador_carros"
)


subreddit = reddit.subreddit("cars")
print(f"Extrayendo publicaciones del subreddit: {subreddit.display_name}")


Extrayendo publicaciones del subreddit: cars


In [4]:
post = reddit.submission(url="https://www.reddit.com/r/cars/comments/1737o1u/whats_your_daily_driver_and_why/")
post.comments.replace_more(limit=0)

print(post)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



1737o1u


In [5]:
comments = [comment.body for comment in post.comments.list()]

# Convertir a DataFrame para análisis
df = pd.DataFrame(comments, columns=["Comentario"])

In [6]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# stopwords y lematizador
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Función de preprocesamiento
def preprocess_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)


df['comentario_limpio'] = df['Comentario'].apply(preprocess_text)
print(df['comentario_limpio'].head())


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0                              need practicality spine
1          hellcat redeye charger im stupid hate moneu
2                                              deleted
3    tacoma cuz hate power fuel economy love huntin...
4                   heavily modified mgb refuse normal
Name: comentario_limpio, dtype: object


In [7]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [8]:
sia = SentimentIntensityAnalyzer()

#funcion para dar puntajes
def analyze_sentiment(text):
    sentiment_scores = sia.polarity_scores(text)
    return sentiment_scores
df['sentiment_scores'] = df['Comentario'].apply(analyze_sentiment)

#coljumnas de puntajes
df['neg'] = df['sentiment_scores'].apply(lambda x: x['neg'])  # Negatividad
df['neu'] = df['sentiment_scores'].apply(lambda x: x['neu'])  # Neutralidad
df['pos'] = df['sentiment_scores'].apply(lambda x: x['pos'])  # Positividad
df['compound'] = df['sentiment_scores'].apply(lambda x: x['compound'])  # Puntaje compuesto

In [9]:
def sentiment_category(compound_score):
    if compound_score >= 0.05:
        return 'Positive'
    elif compound_score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

# Aplicar la categorización
df['sentiment_category'] = df['compound'].apply(sentiment_category)

In [10]:
df[['Comentario', 'sentiment_category']]

,Comentario,sentiment_category
0,My 911 because who needs practicality or a spine.,Neutral
1,A hellcat redeye charger. \nBecause I’m stupid...,Negative
2,[deleted],Neutral
3,"Tacoma cuz I hate power, fuel economy, and lov...",Positive
4,Heavily modified 1977 MGB. Just because I ref...,Negative
...,...,...
482,Autopilot will keep you in the lane better on ...,Positive
483,Ok you have the ability to copy/paste. Very go...,Positive
484,My last one was a 1975 Moto Guzzi 850 police i...,Positive
485,>Why Toyota decided to stick with a 17 year ol...,Negative


In [11]:
# Inicializar recursos de NLTK
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Función de preprocesamiento
def preprocess_for_frequency(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

df['tokens'] = df['Comentario'].apply(preprocess_for_frequency)
all_tokens = [word for tokens in df['tokens'] for word in tokens]

word_frequencies = Counter(all_tokens)
# Mostrar las palabras más comunes
most_common_words = word_frequencies.most_common(10)

#### Analisis

### *Datos*

Se Extrae información de comntarios de post de Reddit relacionado a autos.

In [12]:
comments[0:10]

['My 911 because who needs practicality or a spine.',
 'A hellcat redeye charger. \nBecause I’m stupid and hate moneu',
 '[deleted]',
 'Tacoma cuz I hate power, fuel economy, and love hunting for gears.',
 'Heavily modified 1977 MGB.  Just because I refuse to be normal.',
 'M3 Competition, because I dailied an S2000 all throughout college and that was very much a "never meet your heroes" moment that was ruining my dream car for me. Worst daily driver ever. Now with the M3 as a daily, the S2000 again feels like the baby F1 car that I bought it to be.',
 'Life is too short to not drive something fun on the daily, but I only have room for one car at my house, so all-arounder M car it is.',
 '[deleted]',
 'I take a tram most days.',
 'Ford Focus ST, because hot hatchback practically and speed']

In [13]:
most_common_words

[('car', 272),
 ('daily', 86),
 ('like', 82),
 ('one', 82),
 ('drive', 78),
 ('get', 63),
 ('im', 61),
 ('tesla', 58),
 ('fun', 57),
 ('dont', 57)]

Se utiliza la libreria nltk.sentiment para categorizar los comentarios en funcion de su sentimiento trasmitido


In [14]:
df[['Comentario', 'sentiment_category']]

,Comentario,sentiment_category
0,My 911 because who needs practicality or a spine.,Neutral
1,A hellcat redeye charger. \nBecause I’m stupid...,Negative
2,[deleted],Neutral
3,"Tacoma cuz I hate power, fuel economy, and lov...",Positive
4,Heavily modified 1977 MGB. Just because I ref...,Negative
...,...,...
482,Autopilot will keep you in the lane better on ...,Positive
483,Ok you have the ability to copy/paste. Very go...,Positive
484,My last one was a 1975 Moto Guzzi 850 police i...,Positive
485,>Why Toyota decided to stick with a 17 year ol...,Negative
